In [59]:
import pandas as pd

In [60]:
df = pd.read_csv('raw_data.csv')

In [61]:
dfc = pd.read_csv('cleaned-data.csv')

In [ ]:
print(df.price.isna().sum())
print(dfc.price.isna().sum())

# Dropping entries with missing values in "price" (36)
dfc = df.dropna(subset=['price'], how='all')
dfc.reset_index(inplace=True)

36
0


In [ ]:
# Drop rows where 'living_area' is 0 (5 entries)
dfc = dfc[dfc["living_area"] != 0]

In [ ]:
# Handling missing values in "facade_number" (9362)

# Compute median facade number by subtype
facade_dict = dfc.groupby("subtype_of_property")["facade_number"].median().to_dict()

# Impute values
dfc["facade_number"] = dfc["facade_number"].fillna(dfc["subtype_of_property"].map(facade_dict))

In [ ]:
dfc.drop(['index'], axis=1, inplace=True)
print(dfc)
#

In [58]:
dfc.to_csv('cleaned-data.csv', index=False)